In [8]:
import numpy as np
import open3d as o3d
import copy
import logging

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
import numpy as np
from numba import jit
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon


@jit(nopython=True, cache=True, parallel=True)
def vector_angle(u, v=np.array([0., 0., 1.])):
    """
    Returns the angle in degrees between vectors 'u' and 'v'. If only 'u' is
    provided, the angle between 'u' and the vertical axis is returned.
    """
    # see https://stackoverflow.com/a/2827466/425458
    c = np.dot(u/np.linalg.norm(u), v/np.linalg.norm(v))
    clip = np.minimum(1, np.maximum(c, -1))
    return np.rad2deg(np.arccos(clip))


@jit(nopython=True, cache=True, parallel=True)
def get_octree_level(points, grid_size):
    """Compute nearest octree level based on a desired grid_size."""
    dims = np.zeros((points.shape[1],))
    for d in range(points.shape[1]):
        dims[d] = np.max(points[:, d]) - np.min(points[:, d])
    max_dim = np.max(dims)
    if max_dim < 0.001:
        return 0
    octree_level = np.rint(-np.log(grid_size / max_dim) / (np.log(2)))
    if octree_level > 0:
        return np.int64(octree_level)
    return 1


@jit(nopython=True, cache=True, parallel=True)
def compute_bounding_box(points):
    """
    Get the min/max values of a point list.

    Parameters
    ----------
    points : array of shape (n_points, 2)
        The (x, y) coordinates of the points. Any further dimensions will be
        ignored.

    Returns
    -------
    tuple
        (x_min, y_min, x_max, y_max)
    """
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])

    return (x_min, y_min, x_max, y_max)


def convex_hull_poly(points):
    """Return convex hull as a shapely Polygon."""
    return Polygon(points[ConvexHull(points, qhull_options='QJ').vertices])


def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)電気電子情報工学科
    # Calculate center point and project onto rotated frame
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    center_point = np.dot([center_x, center_y], r)

    min_bounding_rect = np.zeros((4, 2))
    min_bounding_rect[0] = np.dot([x1, y2], r)
    min_bounding_rect[1] = np.dot([x2, y2], r)
    min_bounding_rect[2] = np.dot([x2, y1], r)
    min_bounding_rect[3] = np.dot([x1, y1], r)

    # Compute the dims of the min bounding rectangle
    dims = [(x1 - x2), (y1 - y2)]

    return min_bounding_rect, hull_points, min(dims), max(dims), center_point


In [10]:
pcd = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/aichi-20240412-library2_croped_minimini_1.pcd")

In [11]:
las_labels = np.array([1,1,2,2,2,3,3,3,4,5,8])

In [12]:
mask = np.ones((len(las_labels),), dtype=bool)

In [13]:
mask = mask & (las_labels != 2)
mask

array([ True,  True, False, False, False,  True,  True,  True,  True,
        True,  True])

In [14]:
list_of_indices = np.where(las_labels[mask] == 8)[0]
list_of_indices

array([7])

In [15]:
mask_indices = np.where(mask)[0]
mask_indices

array([ 0,  1,  5,  6,  7,  8,  9, 10])

In [16]:
label_mask = np.zeros(len(mask), dtype=bool)
label_mask

array([False, False, False, False, False, False, False, False, False,
       False, False])

In [72]:
threshold_angle=30 #20
threshold_curve=0.5 #1.0
max_nn=50 #30
grow_region_knn=50 #15
grow_region_radius=0.2

In [73]:
def _compute_point_curvature(coords, pcd_tree, seed_point, method):
    """ Compute the curvature for a given a cluster of points. """
    if method == 'radius':
        _, idx, _ = (pcd_tree.search_radius_vector_3d(
                        seed_point, grow_region_radius))
    else:
        _, idx, _ = (pcd_tree.search_knn_vector_3d(
                        seed_point, grow_region_knn))

    neighbors = o3d.utility.Vector3dVector(coords[idx])
    pcd = o3d.geometry.PointCloud(neighbors)
    _, cov = pcd.compute_mean_and_covariance()
    eig_val, _ = np.linalg.eig(cov)

    return (eig_val[0]/(eig_val.sum()))

In [74]:
# pcd_tree.search_radius_vector_3d(np.array([-70.3690, 12.577,1.001]),0.01)
list_of_seed_ids = [1075605]

In [75]:
region = copy.deepcopy(list_of_seed_ids)
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                                            radius=grow_region_radius,
                                            max_nn=max_nn))

processed = np.full(len(pcd.points), False)
processed[list_of_seed_ids] = True


In [76]:

method = "knn"
idx = 0
while idx < len(list_of_seed_ids):
    print(len(list_of_seed_ids), idx)
    seed_point = pcd.points[list_of_seed_ids[idx]]
    seed_normal = pcd.normals[list_of_seed_ids[idx]]

    # For every seed point, the algorithm finds its neighboring points
    if method == 'radius':
        k, neighbor_idx, _ = (pcd_tree.search_radius_vector_3d(
                                seed_point, grow_region_radius))
    else:
        k, neighbor_idx, _ = (pcd_tree.search_knn_vector_3d(
                                seed_point,grow_region_knn))

    # Remove index seed point itself
    neighbor_idx = neighbor_idx[1:k]

    for neighbor_id in neighbor_idx:
        # Is this point processed before?
        if processed[neighbor_id]:
            continue

        # Compute angles between two n-dimensional vectors
        current_angle = vector_angle(seed_normal,
                                        pcd.normals[neighbor_id])
        # The smoothness constraint in degrees
        if current_angle < threshold_angle:
            region.append(neighbor_id)
            processed[neighbor_id] = True

            # Compute the curvature for a neighbor_id and its neighbors
            curvature = (_compute_point_curvature(
                            np.asarray(pcd.points), pcd_tree,
                            pcd.points[neighbor_id], method))

            # Result is below threshold, we add it to the seed points
            if curvature < threshold_curve:
                list_of_seed_ids.append(neighbor_id)

    idx = idx+1
print(list_of_seed_ids)
print(region)


1 0
8 1
9 2
10 3
13 4
13 5
13 6
15 7
15 8
15 9
21 10
21 11
21 12
23 13
24 14
24 15
35 16
35 17
37 18
38 19
39 20
40 21
42 22
42 23
42 24
45 25
45 26
45 27
45 28
45 29
45 30
46 31
46 32
46 33
46 34
46 35
46 36
47 37
47 38
47 39
49 40
51 41
51 42
51 43
51 44
51 45
51 46
51 47
52 48
52 49
52 50
52 51
53 52
54 53
58 54
58 55
63 56
63 57
64 58
64 59
73 60
79 61
86 62
86 63
86 64
86 65
86 66
86 67
86 68
86 69
86 70
86 71
86 72
88 73
88 74
88 75
88 76
88 77
88 78
88 79
88 80
88 81
88 82
88 83
88 84
88 85
88 86
97 87
97 88
98 89
98 90
98 91
98 92
98 93
98 94
99 95
99 96
99 97
100 98
104 99
105 100
105 101
105 102
106 103
106 104
108 105
109 106
113 107
113 108
113 109
116 110
118 111
121 112
129 113
131 114
131 115
131 116
131 117
131 118
133 119
133 120
133 121
133 122
133 123
133 124
133 125
134 126
134 127
134 128
134 129
134 130
134 131
139 132
143 133
143 134
146 135
146 136
146 137
146 138
146 139
150 140
150 141
150 142
150 143
150 144
150 145
150 146
153 147
154 148
155 149
156 150
156

In [79]:
o3d.visualization.draw_geometries([pcd.select_by_index(region)])

In [80]:
pcd.select_by_index(region)

PointCloud with 914 points.